# Trying to Condense Code

This code opens a webpage and attempts to get every link from within it

Problems with the below code: HTTPError 403, useful information may be found on the pages of links (this can be implemented fairly easily but could increase run time dramatically), still need to handle pdf's 

Thao said that there was a work around for 403's??

In [113]:
from bs4 import BeautifulSoup
import urllib
import re
import lxml.html
 
originalURL = 'https://www.richland2.org/charterhigh/'
html_page = urllib.request.urlopen(originalURL)

soup = BeautifulSoup(html_page, "lxml")

#Need to adjust parent URL to ensure that it can be used to filter the links in the page
def getUsefulParentURL(parentURL):
    #list of possible correct endings for the parentURL
    possibleParentEnds = ['.org', '.edu', '.com']
    
    for end in possibleParentEnds:
        #attempts to split parentURL by one of the endings
        parentURLSplit = parentURL.split(end)
        
        #if the split is successful then the parentURL is reassigned to the string before the split + the ending
        if len(parentURLSplit) > 1:
            parentURL = parentURLSplit[0] + end
            return parentURL
        
parentURL = getUsefulParentURL(originalURL)

#Finds all links in the page that are within the website and not malformed by ensuring the parentURL is in the URL.
def getUsefulLinks():
    links = []
    
    #if soup.find_all('a') != None:
     #   for link in soup.find_all('a'):
      #      if link != None and link.get('href') != None and parentURL in link.get('href'):
       #         #print(link.get('href'))
        #        links.append(link.get('href'))
                
    
    #links.append(originalURL)
    #return links
    #connection = urllib.urlopen(originalURL)
    
    #dom =  lxml.html.fromstring(connection.read())

    #for link in dom.xpath('//a/@href'): # select the url in href for all a tags(links)
     #   print link
    #return links
    pattern = re.compile("((http|ftp)s?://.*?)")
    for child in soup.recursiveChildGenerator():
        name = getattr(child, "name", None)
        if name is not None:
            if name == 'a':
                if child.get('href') != None:
                    current_link = child.get('href')
                    if not pattern.match(current_link):
                        current_link = urlJoin(parentURL, current_link)
                    if parentURL in current_link:
                        links.append(current_link)
    links.append(parentURL)
    return links

In [114]:
def urlJoin(parentURL, end):
    if end[0] == '/':
        return parentURL + end
    else:
        return parentURL + '/' + end

In [115]:
def allChildrenNames():
    pattern = re.compile("((http|ftp)s?://.*?)")
    
    for child in soup.recursiveChildGenerator():
        name = getattr(child, "name", None)
        if name is not None:
            if name == 'a':
                if child.get('href') != None:
                    current_link = child.get('href')
                    if not pattern.match(current_link):
                        current_link = urlJoin(parentURL, current_link)
                    if parentURL in current_link:
                        print(current_link)
        elif not child.isspace(): # leaf node, don't print spaces
            print('')

allChildrenNames()









https://www.richland2.org/CharterHigh/


https://www.richland2.org/CharterHigh/

https://www.richland2.org/CharterHigh/About-Us

https://www.richland2.org/CharterHigh/School-Information

https://www.richland2.org/CharterHigh/School-Information/Graduation-2017

https://www.richland2.org/CharterHigh/School-Information/Lab-Hours

https://www.richland2.org/CharterHigh/School-Information/Calendar

https://www.richland2.org/CharterHigh/School-Information/Frequently-Asked-Questions

https://www.richland2.org/CharterHigh/School-Information/Directions

https://www.richland2.org/CharterHigh/School-Information/School-Board

https://www.richland2.org/CharterHigh/School-Information/School-Improvement-Council

https://www.richland2.org/CharterHigh/Contact-Us


https://www.richland2.org/CharterHigh/News

https://www.richland2.org/CharterHigh/Careers

https://www.richland2.org/CharterHigh/Student-Registration


https://www.richland2.org/CharterHigh/Special-Pages/Site-Translations




https://w

In [116]:
def findKeyWordTextNeighborTesterWithNameCheck(soup, keyWord):
    searchTxt = ''
    tagLst = []
    correctName = ['p', 'li', 'table']
    
    #This loop goes through all the text in every tag. If the text contains a keyword, then it puts that text's tag in the
    #tagLst.
    for elem in soup(text=re.compile(keyWord)):
        tagLst.append(elem.parent)
    
    #This loop goes through each tag and rips the text from that tag. If that tag is a header, then the text from the next 
    #useful tag is ripped instead. All text is saved in searchTxt
    for tag in tagLst:
        if 'h' in tag.name:
            typeTag = type(soup.find('li'))
            current = tag.next_sibling
            while current != None and  not isinstance(current, typeTag) and current.name not in correctName:
                current = current.next_sibling
            if current != None:
                searchTxt = searchTxt + current.text
        else:
            if(tag.name in correctName):
                searchTxt = searchTxt + tag.text
            
    return searchTxt

This method applies the scraping method for each keyword in a list of keywords.

In [117]:
def keyWordTextTester(keyWords, soup):
    foundTxt = ''
    
    for keyWord in keyWords:
        foundTxt = foundTxt + findKeyWordTextNeighborTesterWithNameCheck(soup, keyWord)
    
    return foundTxt

Lists for keywords for each category

In [118]:
mission = [' vision ', 'Vision ', 'our purpose', ' cause ', ' objectives ', ' mission', ' vision: ', ' mission: ', 'Mission']

curriculum = [ 'program', 'methods', 'pedagogy', 'approach', 'model', 'curriculum', 'academics', 'degree']

philosophy = ['beliefs', 'principles', 'creed', 'Values', 'philosophy', 'moral']

history = ['story', 'background', 'founded', 'established']

target = ['gifted', 'at-risk', 'ethnic background', 'target population', 'target audience']

resources = ['ESL', 'tutoring', 'after-school programs', 'available resources', 'services', 'opportunities', 'opportunity']

orgfactor = ['statistics', 'API', 'environment', 'ratio', 'average', 'female', 'fund', 'community']

Once we have the useful links we need to go loop through each one and parse that

In [119]:
#this method takes a list of links and applies the scraping method to that list. 
#WARNING: this method may take a while since it goes through every link, give it time to run before terminating it.
def mapURLs(links, keywords):
    foundTxt = ''
    for link in links:
        try:
            new_page = urllib.request.urlopen(link)
            new_soup = BeautifulSoup(new_page, "lxml")
            foundTxt = foundTxt + keyWordTextTester(keywords, new_soup)
        except urllib.error.HTTPError as err:
            if err.code == 404:
                foundTxt = 'Page not found! (404 Error)' 
            elif err.code == 403:
                foundTxt = 'Access denied! (403 Error)'
            else:
                foundTxt = 'Something happened! Error code: ' + err.code
        
    return foundTxt

#print(mapURLs(links))

In [120]:
school = {'mission' : [], 'curriculum': [], 'philosophy' : [], 'history' : [], 'target' : [], 'resources' : [],
          'organizational_factors' : []}


Next method is purely for timing purposes

In [121]:
def wrapper(func, *args):
    def wrapped():
        return func(*args)
    return wrapped

Code to run the parser for every keyword list and then adding the scraped text to the corresponding array in the school object.
# Warning this code takes a LONG time, don't give up on it.

In [122]:
links = getUsefulLinks()

def addTextToSchoolObject(school):
    keywordLists = [mission, curriculum, philosophy, history, target, resources, orgfactor]
    schoolArrayNames = ['mission', 'curriculum', 'philosophy', 'history', 'target', 'resources' , 'organizational_factors']

    for x in range(0, len(keywordLists)):
        school[schoolArrayNames[x]] += [mapURLs(links, keywordLists[x])]
        

In [123]:
import timeit

wrapped = wrapper(addTextToSchoolObject, school)
timeit.timeit(wrapped, number=1)

112.27559246280498

yikes that's a long time...

In [124]:
print(school)

{'mission': ['Page not found! (404 Error)'], 'curriculum': ['Page not found! (404 Error)'], 'philosophy': ['Page not found! (404 Error)'], 'history': ['Page not found! (404 Error)'], 'target': ['Page not found! (404 Error)'], 'resources': ['Page not found! (404 Error)Richland School District Two is fully committed to providing information and services to the community that meet a variety of communication needs. \xa0The district is continuously engaged in making improvements to this site to ensure everyone in our community has equal access to the tools and information provided on this site.'], 'organizational_factors': ['Page not found! (404 Error)Richland School District Two is fully committed to providing information and services to the community that meet a variety of communication needs. \xa0The district is continuously engaged in making improvements to this site to ensure everyone in our community has equal access to the tools and information provided on this site.In partnership wi